# Jarvis march algorithm

Use the cell below for all python code needed to realise the Jarvis march algorithm (including any auxiliary data structures and functions you might need). The `jarvismarch()` function itself should take as input parameter a list of 2D points (`inputSet`), and return the subset of such points (`outputSet`) that lie on the convex hull.

In [1]:
def leftmost(points: list) -> tuple:
    """Function that fetches the leftmost point from a list of points.

    Args:
        points (list): a list of di-tuples that represent co-ordinates (x,y)

    Returns:
        (tuple): a di-tuple that represents that leftmost point from the list of points.
    """
    left = points[0]
    for point in points:
        if point[0] < left[0]:
            left = point
    return left

def determinant(p1: tuple, p2: tuple, p3: tuple) -> float:
    """Function that gets the determinant for p1, p2 and p3, to determine whether it is a clockwise or
       anticlockwise turn. 

        Args:
        p1 (tuple) : a di-tuple (x,y) representing the first coordinate,
        p2 (tuple) : a di-tuple (x,y) representing the second coordinate,
        p3 (tuple) : a di-tuple (x,y) representing the third coordinate,

    Returns:
        (float) : the determinant between p1, p2 and p3.

    Note:
        det > 0 -> clockwise turn
        det < 0 -> anticlockwise turn
        det == 0 -> points are collinear
    """

    return (p2[0]-p1[0]) * (p3[1]-p1[1])  - (p2[1]-p1[1]) * (p3[0]-p1[0])

In [2]:
def jarvismarch(inputSet: list):
    """Function that uses the Jarvis March algorithm to get the coordinates that are on the hull.

    args:
        inputSet (list): list containing pairs of x,y coords.

    returns:
        (list) containing the points on the hull
    """
    left = leftmost(inputSet) # get the left point
    pointOnHull = left # leftmost point is definitely on the hull, and is placed as starting point.
    outputSet = []
    i = 0
    endpoint = None
    while endpoint != left: # repeat until we reach the start point.
        outputSet.append(pointOnHull) # add point to the list of points that are on the convex hull
        endpoint = inputSet[0] # set endpoint to the start of the list.
        for point in inputSet:
            # loop through the set of points given.
            if determinant(outputSet[i], endpoint, point) > 0 or (endpoint == pointOnHull) :
                # if a point is on the left of the line, then take that point.
                endpoint = point			
        i += 1
        pointOnHull = endpoint
    return outputSet

Use the cell below for all python code needed to generate test data points (both random and those representing worst-case scenario).

In [3]:
import random

#code for random data generation
def generatePoint() -> tuple:
    """Function that generates a random valid point"""
    return (random.randint(0, 32767), random.randint(0, 32767))

def generatePoints(limit: int) -> list:
    """Function that generates a random list of point"""
    return [generatePoint() for point in range(limit)]

def generateFloat() -> tuple:
    """Function that generates a random valid point"""
    return (random.randint(0, 32767)*random.random(), random.randint(0, 32767)*random.random())

def generateFloats(limit: int) -> list:
    """Function that generates a random list of point"""
    return [generateFloat() for point in range(limit)]

#code for worst case data generation

def getCirclePoint(number: int, centre: tuple, radius: int) -> tuple:
    """Function that produces a point on the circle given its arguments.

    Args:
        number (int): a number from 0 -> limit,
        centre (tuple): a di-tuple representing the centre point coordinate of the circle,
        radius (integer): an integer representing the radius of the circle

    Returns:
        (tuple): a di-tuple coordinate (x,y) that represents a point on the circle.
    """
    # radius^2 = (x - centre[0])^2 + (y - centre[1])^2
    # radius^2 - (x - centre[0]) ^ 2 = (y - centre[1])^2
    # sqrt(radius^2 - (x - centre[0])^2) + centre[1] = y
    x = centre[0] - radius + number
    y = (((radius**2) - (x - centre[0])**2)**0.5 + centre[1]) *  ((-1)**(number%2)) + radius
    return [x, y]

def worstCase(limit: int) -> list:
    """Function that produces the worst case for Jarvis March, which is when all,
    the points are on the convex hull, i.e. the points form a circle.

    Args:
        limit (int): the number of points to generate

    Returns:
        (list): Returns a list of di-tuples that represent coordinates on a circle.
    """
    centre_point = [16383,0]
    return [getCirclePoint(point, centre_point, limit//2) for point in range(limit)]


Use the cell below for all python code needed to test the `jarvismarch()` function on the data generated above.

In [4]:
import timeit

#test code

def run_tests(limits: list, data_generator, title="", tests=100):
    """Procedure that tests the code, and presents the findings in a prettified manner.

    Args:
        limits (list): a list of limits to test the jarvismarch on.
        data_generator (function): a function, that when given a limit from limits, generates a list of points.
        title (str): title is used as a label
        tests (int): number of times to run a limit. Defaults to 100.
    """
    
    print("="*10, f" STARTING: {title} cases ", "="*10, "\n")
    for limit in limits:
        # take avg. data generation time to subtract from total time, to get time taken for jarvismarch.
        data_generation_time = timeit.timeit('data_generator(limit)', number = tests, globals={'data_generator':data_generator, 'limit':limit})

        # get the avg. time taken to jarvismarch a generated list (a new list is generated each iteration), includes time for data generation.
        total_time = timeit.timeit('jarvismarch(data_generator(limit))', number = tests,  
        globals={'jarvismarch':jarvismarch,'data_generator':data_generator, 'limit':limit})

        # subtract total_time by the time it takes to generate data, and then take avg.
        jarvismarch_time = (total_time - data_generation_time) / tests
        
        string =  f'{title} Case @ {limit}: {jarvismarch_time} seconds' 
        print(string)

    print("\n", "="*10, f" FINISHED: {title} cases ", "="*10)

limits = [100, 500, 1000, 5000, 10000, 15000, 20000]
run_tests(limits, generatePoints, "Normal")
run_tests(limits, worstCase, "Worst", tests=5)



==========  STARTING: Normal cases  ========== 

Normal Case @ 100: 0.0010543170000000136 seconds
Normal Case @ 500: 0.007754028000000011 seconds
Normal Case @ 1000: 0.017344457 seconds


KeyboardInterrupt: 

*Optional*: Feel free to use the code below on small datasets (e.g., N = 10) to visually inspect whether the algorithm has been implemented correctly. The fragment below assumes both `inputSet` and `outputSet` to be lists of 2D points, with each point being a list of 2 elements (e.g., `[[x1,y1], [x2,y2], ..., [x_k,y_k]]`)

In [ ]:
import matplotlib.pyplot as plt

# inputSet and outputSet should have been defined above. 
# uncomment the next two lines only if you wish to test the plotting code before coding your algorithm

inputSet = [[1,1], [2,2] , [3, 3], [4,4], [1,4], [3,1], [1, 5], [2, 4], [3, 5]]
outputSet = [[1,1], [3,1] , [4, 4], [3,5], [1,5]]

inputSet = worstCase(7)
outputSet = jarvismarch(inputSet)

plt.figure()

#first do a scatter plot of the inputSet
input_xs, input_ys = zip(*inputSet)
plt.scatter(input_xs, input_ys)

#then do a polygon plot of the computed covex hull
outputSet.append(outputSet[0]) #first create a 'closed loop' by adding the first point at the end of the list
output_xs, output_ys = zip(*outputSet) 
plt.plot(output_xs, output_ys) 

plt.show() 